Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1178]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue

import numpy as np

In [1179]:
PROBLEM_SIZE = 10
NUM_SETS = 10
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)
)

State = namedtuple("State", ["taken", "not_taken"])

In [1180]:
# find if exist a possible solution
def goal_check(state):      
    return np.all(total_state(state.taken))

# find how many false it takes
def distance(state):         
    return PROBLEM_SIZE - sum(state)


# compute logical or between all the set taken in the current state
def total_state(state):     
    return reduce(
        np.logical_or,
        [SETS[i] for i in state],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


# key function to sort the untaken set of the current state using the amount of overlapping with the taken set and
# the distance of the untaken set 
def sorting_for_overlapping(e, state):      
    return sum(np.logical_and(state, SETS[e]))+ distance(SETS[e])


def function(current_state):
    tot_current_state = total_state(current_state.taken)

    sorted_untaken = sorted(
        current_state.not_taken, key=lambda e: sorting_for_overlapping(e, tot_current_state)
    )

    h = 0
    # h is the number of untaken sets necessary to cover all the remaining false of the current state
    # the untaken sets were sorted, so if one of them can fill the state, it will be the first of the list
    for action in sorted_untaken:
        if distance(tot_current_state) == 0:
            break
        tot_current_state = np.logical_or(tot_current_state, SETS[action])
        h += 1


    # h is my heuristic, the g of the A* algorithm is the number of taken state
    return h + len(current_state.taken)

In [1181]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

In [1182]:
# sort SETS considering the number of false of each single set
def sorting_key(e):
    return e.sum()


def get_ordered_sets():
    return sorted(SETS, reverse=True, key=sorting_key)

In [1183]:
# function to force some blocks if these are the only ones which cover some True position
def get_mandatory_blocks():
    unic_list = np.vstack(SETS).sum(0)

    if 1 not in unic_list:
        return State(set(), set(range(NUM_SETS)))

    ind = [
        index
        for index, s in enumerate(SETS)
        if any(unic_list[i] == 1 and s[i] == True for i in range(PROBLEM_SIZE))
    ]
    return State(set(ind), set(range(NUM_SETS)) - set(ind))

In [1184]:
frontier = PriorityQueue()
SETS = get_ordered_sets()
state = get_mandatory_blocks()

frontier.put((function(state), state))


_, current_state = frontier.get()
counter = 0 
solution = None if not goal_check(current_state) else current_state


while solution is None:
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
 
        frontier.put((function(new_state), new_state))
        
        if distance(total_state(new_state.taken)) == 0:
            solution = new_state
            break
    _, current_state = frontier.get()



print(f"Solved in {counter:,} steps ({len(solution.taken)} tiles)")

Solved in 1 steps (4 tiles)


In [1185]:
solution.taken

{0, 1, 2, 3}